In [2]:
from scipy.ndimage import binary_dilation, binary_erosion, zoom
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
import os

In [3]:
class PictureDamager:
    def __init__(self, masks: np.array, stain_percentage: float):
        self.stain_percentage = stain_percentage
        self.masks = masks

    def expected_num_of_stains(self, mask_size):
        return mask_size[0] * mask_size[1] * self.stain_percentage

    def mask_randomizer(self, mask, resize_prob, morph_transform_prob):
        if random.random() < resize_prob:
            enlarge = random.random() < 0.5
            if enlarge:
                if random.random() < 0.5:
                    mask = zoom(mask, (2, 2), order=0)
                else:
                    mask = zoom(mask, (3, 3), order=0)
            else:
                mask = zoom(mask, (0.5, 0.5), order=0)

        if random.random() < morph_transform_prob:
            dilate = random.random() < 0.5
            if dilate:
                mask = binary_dilation(mask).astype(np.uint8)
            else:
                mask = binary_erosion(mask).astype(np.uint8)

        if random.random() < 0.5:
            mask = mask.T

        if random.random() < 0.5:
            mask = np.flipud(mask)

        if random.random() < 0.5:
            mask = np.fliplr(mask)

        return np.array(mask)

    def generate_random_mask(self, mask_size):
        damaged_mask = np.zeros(mask_size, dtype=np.uint8)
        stains_counter = 0

        while stains_counter < self.expected_num_of_stains(mask_size):
            mask = random.choice(self.masks)
            mask = self.mask_randomizer(mask, 0.5, 0.5)

            mask_height, mask_width = mask.shape
            if mask_height > mask_size[0] or mask_width > mask_size[1]:
                continue

            x = random.randint(0, mask_size[0] - mask_height)
            y = random.randint(0, mask_size[1] - mask_width)

            damaged_mask[x:x+mask_height, y:y+mask_width] = np.where(
                mask == 1, 1, damaged_mask[x:x+mask_height, y:y+mask_width])

            stains_counter += np.sum(mask)

        return damaged_mask

    def add_masks_to_images(self, image_folder, output_folder, target_size=(256, 256)):
        os.makedirs(output_folder, exist_ok=True)

        for image_file in os.listdir(image_folder):
            if image_file.endswith('.png') or image_file.endswith('.jpg'):
                image = np.array(Image.open(os.path.join(image_folder, image_file)))

                if image.shape[-1] != 3:
                    print(f"Skipping {image_file} (not a 3-channel image)")
                    continue

                if image.shape[:2] != target_size:
                    print(f"Resizing image {image_file} from {image.shape[:2]} to {target_size}")
                    image = np.array(Image.fromarray(image).resize(target_size, Image.ANTIALIAS))

                mask = self.generate_random_mask(target_size)

                image_with_mask = np.dstack((image, mask))

                output_file = os.path.splitext(image_file)[0] + ".npy"
                np.save(os.path.join(output_folder, output_file), image_with_mask)

        print(f"Processed images saved to {output_folder}")

    def generate_damaged_images(self, image_folder_with_masks, output_folder):
        os.makedirs(output_folder, exist_ok=True)

        for file in os.listdir(image_folder_with_masks):
            if file.endswith('.npy'):
                image_with_mask = np.load(os.path.join(image_folder_with_masks, file))
                image, mask = image_with_mask[:, :, :3], image_with_mask[:, :, 3]

                damaged_image = np.where(mask[..., None] == 1, 255, image)

                output_file = os.path.splitext(file)[0] + "_damaged.png"
                Image.fromarray(damaged_image.astype(np.uint8)).save(os.path.join(output_folder, output_file))

        print(f"Damaged images saved to {output_folder}")

In [4]:
masks_path = "masksnpy"
MASKS = [np.load(os.path.join(masks_path, f)) for f in os.listdir(masks_path) if f.endswith(".npy")]  

In [6]:
damager = PictureDamager(MASKS, 1/16)